In [ ]:
import constants
import pandas  as pd
from tqdm import tqdm
from pandarallel import pandarallel
import numpy as np
import scipy.stats
import time
import random
import os
import collections
import json

In [ ]:
import importlib
importlib.reload(constants)

In [ ]:
def throttle(quota):
    def decorator(f):
        t_last = 0
        def nested(*args,**kwargs):
            nonlocal t_last
            t_now = time.time()
            diff = t_now - t_last
            if diff < quota:
                time.sleep(quota - diff)
            t_last = t_now
            f(*args,**kwargs)
        return nested
    return decorator
    

In [ ]:
tqdm.pandas()
pandarallel.initialize()

In [ ]:
df_population = pd.read_csv(constants.CENSUS_2019_ZCTA_POPULATION,sep=",",header=[1]).rename(columns=lambda x: x.strip())
df_gazetteer = pd.read_csv(constants.CENSUS_2019_GAZETTEER,sep="\t").rename(columns=lambda x: x.strip())
df_income = pd.read_csv(constants.CENSUS_2019_ZCTA_INCOME,sep=",",header=[1]).rename(columns=lambda x: x.strip()).rename(columns=lambda x: x.strip())

In [ ]:
df_population["ZCTA5"] = df_population["Geographic Area Name"].str.slice(-5)
df_income["ZCTA5"] = df_income["Geographic Area Name"].str.slice(-5)
df_gazetteer["ZCTA5"] = df_gazetteer["GEOID"].astype("str").str.zfill(5)

df_population = df_population.set_index("ZCTA5")
df_income = df_income.set_index("ZCTA5")
df_gazetteer = df_gazetteer.set_index("ZCTA5")

In [ ]:
df_population["median_income"] = df_income["Estimate!!Families!!Median income (dollars)"].str.replace("+","").str.replace(",","").map(lambda x: "0" if x == "-" or x is None else x).str.replace("-","").astype("int")

In [ ]:
df_population["median_income"]

In [ ]:
df_population["area"] = df_gazetteer.ALAND
df_population["density"] = df_population["Estimate!!Total"] / df_population.area

In [ ]:
%%script false --no-raise-error
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="census_zip_location_data")

@throttle(1)
def get_state(row):
    return geolocator.reverse(f"{row.INTPTLAT},{row.INTPTLONG}").raw["address"]["state"]
df_gazetteer["state"] = df_gazetteer.progress_apply(get_state,axis=1)
df_population["state"] = df_gazetteer.state

In [ ]:
%%script false --no-raise-error
state_density_data = df_population.groupby("state").apply(lambda df: df["Estimate!!Total"].sum()/df.area.sum())
state_density_data.sort_values()

In [ ]:
%%script false --no-raise-error
scipy.stats.spearmanr(df_population.density, df_population["Estimate!!Total"])

# Stratify

In [ ]:
n_strata = 5 #Number of strata for each study

In [ ]:
df_population = df_population.sort_values("density",ascending=False)
n_zips = len(df_population)

pop_count = df_population["Estimate!!Total"].sum()
print(pop_count)

strata_density = [None] * n_strata

stratum_start = 0
zip_loc = 0
for i in range(n_strata):
    print(i)
    zip_start = zip_loc
    stratum_end = stratum_start
    stratum_goal = int((i+1) * pop_count / n_strata)
    while stratum_end < stratum_goal:
        stratum_end += df_population["Estimate!!Total"].iloc[zip_loc]
        zip_loc += 1
    print(f"""Population of stratum: {stratum_end - stratum_start} (range: {stratum_start} - {stratum_end}, end goal: {stratum_goal})
Density range: {df_population.density[zip_loc]} {df_population.density[zip_start]}
Number of zipcodes: {zip_loc-zip_start} ({zip_start} - {zip_loc})
          """)
    
    strata_density[i] = df_population.iloc[zip_start:zip_loc].sample(frac=1).index.array
    
    stratum_start = stratum_end

In [ ]:
df_population = df_population.sort_values("median_income",ascending=False)
n_zips = len(df_population)

pop_count = df_population["Estimate!!Total"].sum()
print(pop_count)

strata_income = [None] * n_strata

stratum_start = 0
zip_loc = 0
for i in range(n_strata):
    print(i)
    zip_start = zip_loc
    stratum_end = stratum_start
    stratum_goal = int((i+1) * pop_count / n_strata)
    while stratum_end < stratum_goal:
        stratum_end += df_population["Estimate!!Total"].iloc[zip_loc]
        zip_loc += 1
    print(f"""Population of stratum: {stratum_end - stratum_start:,d} (range: {stratum_start:,d} - {stratum_end:,d}, end goal: {stratum_goal:,d})
Median income range: {df_population.median_income[zip_loc]:,d} {df_population.median_income[zip_start]:,d}
Number of zipcodes: {zip_loc-zip_start:,d} ({zip_start:,d} - {zip_loc:,d})
          """)
    
    strata_income[i] = df_population.iloc[zip_start:zip_loc].sample(frac=1).index.array
    
    stratum_start = stratum_end

In [ ]:
sampled_zip_groups = []

In [ ]:
rand = random.Random(0)
pop_goal = 100000
for i in range(len(strata_density)):
    rand_local = random.Random(1+i)
    l1 = list(rand.sample(list(strata_density[i]),10))
    ls = [list(rand_local.sample(list(strata_density[i]),20)) for _ in range(10)]
    
    sampled_zip_groups.append(l1+sum(ls,[]))
    s = "\n\t" + "\n\t".join((' '.join(l) for l in [l1]+ls))
    print(f"{i}: {s}")

In [ ]:
rand = random.Random(0)
pop_goal = 100000
for i in range(len(strata_income)):
    rand_local = random.Random(1+i)
    l1 = list(rand.sample(list(strata_income[i]),10))
    ls = [list(rand_local.sample(list(strata_income[i]),20)) for _ in range(10)]
    
    sampled_zip_groups.append(l1+sum(ls,[]))
    s = "\n\t" + "\n\t".join((' '.join(l) for l in [l1]+ls))
    print(f"{i}: {s}")

In [ ]:
list(map(len,sampled_zip_groups))

In [ ]:
%%script echo skipping
samples="63108 90221 12203 50324 50131 25064 95020 95358 96763 82515 46219 97015 62570 72202 13677 17552 24641 23337 21624 03279 37030 77475 60302 94401 62293 48375 46637 74534 64060 12168 84067 55940 55420 40370 41367 13212 44057 76058 08759 14530 78956 85901 33770 72437 27549 44657 75941 87933 71659 23398 68971 73011 86436 27917 30573 73050 95606 29936 72070 18810 97818 28786 15532 39553 13215 21076 18517 95120 71461 21076 18517 95120 71461 32804 23219 73118 29662 97223 30360 19001 05439 33161 33334 11552 19033 02129 02143 95746 19562 17050 27605 11715 33432 02190 73034 60511 61880 40077 31210 91941 34119 50632 01346 94565 59462 15834 49028 80540 80443 87016 17211 13145 49088 27845 12788 10975 68660 21757 62807 83001 58251 70441 66845 72322 66049 05657 32583 71913 97116 45760 78578 52066 03036 23068 27201 95650 77032 12051 44815 35117 46819 46060 01740 21102 78226 61874 60464 30327 33534 77701 46123 52241 15241 32904 78543 02359 46319 73034 08053 62046 07045 19803 15752 30141 65281 80910 88044 21714 38122 93257 41528 16146 38382 78943 40110 15338 44507 31315 33872 87521 23224 72513 25209 62822 17885 72624 26224 47597 48473 64667 61270 56244 42266 46319 39461 03256 37310 49287 46635 47711 67146 28451 49015 14522 13132 98146 12831 55312 33708 08554 65054 80863 17111 55092 03467 17543 32776 55118 82081 55803 20106 29910 98465 24503 58327 93603 98074 03824 06067 01462 07604 88324 53005 06089 60029 08820 06525 06793 78746 19335 44333 12582 07079 97068 33785 85044 75088 60429 98032 80920 19046 95864 48322 52801 77021 27519 94568 34104 06614 33764 21208 66216 46806 33426 46155 20877 77060 10703 91007 33183 55412 90211 37201 44101 90712 90047 07605 17576 60534 91101 92840 10707 98207 10552 57001 24328 49031 59914 68318 49093 61848 48878 52048 55006 45819 49601 73529 96770 78609 50514 39082 04952 63460 31061 48095 62028 20601 18642 64507 94517 93505 19975 49080 94973 30620 68801 61938 17579 28470 49302 17075 19930 15315 12977 04254 75140 65355 68752 08329 39092 48370 13642 31560 56139 89832 58752 31331 70631 49013 27355 62365 41257 66801 31089 72168 39455 87063 95311 73743 24944 58474 98611 93251 74035 54727 72327 81416 23437 50455 79027 72346 67110 62626 50681 33569 94515 12972 03225 58276 10562 49270 84084 84071 48220 29707 02642 92028 98338 60449 06019 92867 17112 91708 23938 15211 43783 47042 54141 14040 53941 75117 52136 52552 14009 53510 57243 53016 59808 90660 46113 55702 47528 68932 52147 60459 71291 54484 64016 66838 22834 23608 56590 50575 78950 85390 85935 44640 61256 64431 70031 32024 15131 14784 15464 35747 14138 75956 62349 17944 43351 71407 16342 44601 40464 76827 47355 96770 55785 21871 31069 23148 29653 70375 68879 43557 68417 56158 65807 46940 42748 85354 80749 16659 67016 78066 56248 28114 50625 61420 60165 30340 56091 71602 68104 62018 32621 76443 43768 39730 35592 16672 00741 99557 02553 98565 38050 28453 79403 37752 74837 30521 41149 06160 76457 31409 97453 36033 78211 04464 63370 62449 25674 36456 93530 30805 97411 36509 88034 80473 19121 42236 81084 75638 64428 64848 16611 14418 68730 38666 05488 48626 06750 14872 39862 44681 56083 13756 50149 68745 52567 49425 56042 55072 64470 49779 13826 56572 45775 97459 83272 03585 98068 59837 28340 88430 28327 28168 46186 36042 68346 24175 16750 25840 96028 61880 88114 69169 74458 72821 25854 99756 79250 45650 42275 83251 12153 68834 88029 52540 25444 87552 37339 12961 20609 87828 51541 69333 99139 47246 11959 87357 70660 03448 58413 67650 36913 99546 77475 84316 43570 24280 38454 28761 89316 54414 47125 43435 16223 05156 15853 86438 46036 84536 05252 05773 31568 93437 66728 83657 70643 62325 55748 95417 13140 38504 61042 51201 15686 01264 33924 56659 50073 13317 57649 12193 64066 23138 25160 67564 26425 25876 75173 36758 57345 79230 36064 50466 51007 84526 97455 38927 66441 41385 05464 38635 51649 95714 42328 13666 43822 52530 72472 36550 39439 64848 16611 14418 68730 38666 05488 48626 06750 14872 39862 44681 56083 13756 50149 68745 52567 49425 56042 55072 64470 49779 13826 56572 45775 97459 83272 03585 98068 59837 28340 88430 28327 28168 46186 36042 68346 24175 16750 25840 96028 61880 88114 69169 74458 72821 25854 99756 79250 45650 42275 83251 12153 68834 88029 52540 25444 87552 37339 12961 20609 87828 51541 69333 99139 47246 11959 87357 70660 03448 58413 67650 36913 99546 77475 84316 43570 24280 38454 28761 89316 54414 47125 43435 16223 05156 15853 86438 46036 84536 05252 05773 31568 93437 66728 83657 70643 62325 55748 95417 13140 38504 61042 51201 15686 01264 33924 56659 50073 13317 57649 12193 64066 23138 25160 67564 26425 25876 75173 36758 57345 79230 36064 50466 51007 84526 97455 38927 66441 41385 05464 38635 51649 95714 42328 13666 43822 52530 72472 36550 39439 68842 48433 14905 46157 92070 62257 17777 44001 77857 56750 87114 24401 64019 45830 15644 54479 84015 14561 38030 43545 74430 24556 38482 68328 14094 55397 82837 53522 15957 47557 56535 38023 56039 46303 19605 12189 15445 57042 67342 84128 17518 51365 68644 61735 67871 50218 89440 33594 58571 78953 54817 61491 76148 04253 56515 37066 29127 96774 11416 93036 16201 06382 30281 48442 05759 93445 46635 95833 43029 14803 70755 89008 07001 67118 12137 33707 79506 68658 52315 95470 48871 24127 77418 54557 50841 17563 73132 04757 95843 55798 30281 30667 62424 68765 12823 62890 27973 56592 27212 68401 12176 39359 74073 58756 15235 18424 62830 49065 84074 77857 50658 97756 95367 70711 37086 54113 05833 32569 37066 80235 61006 61491 04260 98261 73024 52064 30253 29505 61537 67450 49348 56332 04543 80828 24179 35983 45316 47469 68957 93653 89147 17829 48239 75654 26287 66441 67623 52565 27909 66724 78609 04267 69221 35673 14585 65657 15033 36255 61081 49862 60424 46501 63074 06708 16749 33193 84072 77568 50103 16435 46044 80729 47983 15741 44714 03864 15449 56276 75144 56296 52048 22940 71730 66776 30331 66544 33570 59405 16935 67735 52738 98337 76483 43206 56663 95690 75050 48454 83686 18634 26571 62522 58760 78609 64857 99169 66716 53806 54149 43802 73737 40923 92544 16342 34736 28681 55030 41071 64664 59351 80653 97818 28421 76453 04939 74075 23307 46819 53550 08861 79553 37337 32040 15611 71602 76354 42171 74017 46393 47977 43055 70125 17847 77053 14815 15732 37303 21740 37754 66840 30655 13601 32567 73002 35614 32720 83324 98661 23055 47950 80011 62076 17832 59088 98320 25534 59076 47330 96107 46323 52166 98837 17813 16406 75943 44035 27317 47514 23947 68628 50426 43805 47230 43462 40915 39362 83354 20228 75501 29067 79259 29474 11207 19523 24801 31408 33867 77458 40740 57623 26560 76301 36276 38152 80131 26452 60628 25696 98834 89026 79345 70372 92061 72555 87315 04648 63758 76452 92311 68970 06706 90028 81330 25139 75977 25920 38370 41650 63853 41838 72466 83011 21853 48507 12530 87552 60432 39217 00627 57552 38940 22730 10112 36569 39483 71835 15767 44502 26656 15112 16829 32438 81054 33055 30439 68108 81003 25703 87410 76821 89020 76652 72438 42326 66105 15713 62017 26342 42717 66612 75253 82640 97469 83443 41831 35234 81052 48342 34788 90033 37178 92233 99774 41835 15436 32140 28017 92242 37240 78046 41701 19120 86011 64054 23954 27925 24312 72926 10271 00936 25109 16943 77453 93268 10154 38310 17120 70801 24880 41143 24226 26437 74761 76854 65633 48770 96134 41568 71459 49098".split(" ")

In [ ]:
%%script echo skipping
strata_income_sets = [set(strata_income[i]) for i in range(len(strata_income))]
strata_density_sets = [set(strata_density[i]) for i in range(len(strata_density))]

In [ ]:
%%script echo skipping
sampled_zip_groups = [[] for i in range(len(strata_income)+len(strata_density))]
for zipcode in samples:
    for i in range(len(strata_income)):
        if zipcode in strata_income_sets[i]:
            sampled_zip_groups[i].append(zipcode)
            break
    for i in range(len(strata_density)):
        if zipcode in strata_density_sets[i]:
            sampled_zip_groups[i+len(strata_income)].append(zipcode)
            break

In [ ]:
%%script echo skipping
list(map(len,sampled_zip_groups))

# Selecting samples

In [ ]:
business_file = f"{constants.BUSINESSES_DIR}/%s.json"

In [ ]:
def load_businesses(zipcode, loaded_ids):
    if not os.path.exists(business_file % zipcode):
        #print(f"Not found: {zipcode}")
        return
    with open(business_file % zipcode) as f:
        zipcode_data = json.load(f)
    for business in zipcode_data[zipcode]:
        bid = business["id"]
        if bid not in loaded_ids:
            loaded_ids.add(bid)
            yield business

In [ ]:
bids = set()
businesses = []
for zipcodes in sampled_zip_groups:
    missing = 0
    for zipcode in tqdm(zipcodes,total=len(zipcodes)):
        if not os.path.exists(business_file % zipcode):
            missing += 1
            continue
        for business in load_businesses(zipcode, bids):
            businesses.append(business)
            
    print(f"Missing {missing}/{len(zipcodes)}")

In [ ]:
len(businesses)

In [ ]:
businesses_by_zipcode = collections.defaultdict(list)
for business in businesses:
    businesses_by_zipcode[business["location"]["zip_code"]].append(business)

In [ ]:
try:
    os.mkdir("../../data/businesses_by_zipcode/")
except:
    print("exists")
    pass
try:
    os.mkdir("../../data/not_recommended_reviews/")
except:
    print("exists")
    pass
try:
    os.mkdir("../../data/recommended_reviews/")
except:
    print("exists")
    pass

In [ ]:
for zipcodes in sampled_zip_groups:
    print("--------")
    business_cap = 1000
    business_ct = 0
    for zipcode in zipcodes:
        business_ct += len(businesses_by_zipcode[zipcode])
    print(business_ct)

In [ ]:
total_businesses = 0
total_zips = 0
chosen_zips = []
for zipcodes in sampled_zip_groups:
    print("--------")
    business_cap = 500
    business_ct = 0
    zip_ct = 0
    for zipcode in zipcodes:
        if len(businesses_by_zipcode[zipcode]) == 0:
            print(f"WARNING: {zipcode} is empty")
            continue
        zip_ct += 1
        business_ct += len(businesses_by_zipcode[zipcode])
        chosen_zips.append(zipcode)
        with open("../../data/businesses_by_zipcode/%s.json" % zipcode, "w+") as f:
            json.dump(businesses_by_zipcode[zipcode], f)
        if business_ct >= business_cap:
            print(f"Saved {business_ct} businesses across {zip_ct} zipcodes")
            break
    total_businesses += business_ct
    total_zips += zip_ct
    if business_ct < business_cap:
        print(f"WARNING: only got {business_ct} businesses; expected {business_cap}")
        
print(f"Total {total_businesses} businesses from {total_zips} zipcodes")
print("------------------------------")
print(" ".join(chosen_zips))

In [ ]:
len(chosen_zips)

In [ ]:
rand = np.random.default_rng(12345)
for row in rand.choice(chosen_zips, size=(3,71), replace=False):
    print(" ".join(row))